# United States Hate Crime Relational Analysis

### The following script contains:
#### 01. Importing Data and Libraries
#### 02. Data Wrangling
#### 03. Consistency Checks
#### 04. Choropleth Map Creation

### 01. Importing Data and Libraries

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [6]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [7]:
# Create path and import data
path = r'C:\Users\Jamie&Ken\Desktop\Data Analytics Course\Advanced Analytics & Dashboard Design\10-2024 American Hate Crime'
country_geo = r'C:\Users\Jamie&Ken\Desktop\Data Analytics Course\Advanced Analytics & Dashboard Design\10-2024 American Hate Crime\02 Data\Original Data\us-states.json'
df = pd.read_csv(os.path.join(path,'02 Data','Prepared Data', 'hate_crime_wrangled.csv'))

In [8]:
# Confirm Import
df.head()

INCIDENT_ID  DATA_YEAR        ORI PUB_AGENCY_NAME AGENCY_TYPE_NAME  \
0         3015       1991  AR0040200          Rogers             City   
1         3016       1991  AR0290100            Hope             City   
2           43       1991  AR0350100      Pine Bluff             City   
3           44       1991  AR0350100      Pine Bluff             City   
4         3017       1991  AR0350100      Pine Bluff             City   

  STATE_ABBR STATE_NAME       DIVISION_NAME REGION_NAME POPULATION_GROUP_CODE  \
0         AR   Arkansas  West South Central       South                     5   
1         AR   Arkansas  West South Central       South                     6   
2         AR   Arkansas  West South Central       South                     3   
3         AR   Arkansas  West South Central       South                     3   
4         AR   Arkansas  West South Central       South                     3   

   ... INCIDENT_DATE TOTAL_OFFENDER_COUNT              OFFENDER_RACE  \
0  ...     8/31/1991                    1                      White   
1  ...     9/19/1991                    1  Black or African American   
2  ...      7/4/1991                    1  Black or African American   
3  ...    12/24/1991                    1  Black or African American   
4  ...    12/23/1991                    1  Black or African American   

  VICTIM_COUNT                                       OFFENSE_NAME  \
0            1                                       Intimidation   
1            1                                     Simple Assault   
2            1                                 Aggravated Assault   
3            2  Aggravated Assault;Destruction/Damage/Vandalis...   
4            1                                 Aggravated Assault   

                        LOCATION_NAME                       BIAS_DESC  \
0  Highway/Road/Alley/Street/Sidewalk  Anti-Black or African American   
1  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
2                      Residence/Home  Anti-Black or African American   
3  Highway/Road/Alley/Street/Sidewalk                      Anti-White   
4                 Service/Gas Station                      Anti-White   

  VICTIM_TYPES MULTIPLE_OFFENSE MULTIPLE_BIAS  
0   Individual                S             S  
1   Individual                S             S  
2   Individual                S             S  
3   Individual                M             S  
4   Individual                S             S  

[5 rows x 21 columns]

In [9]:
df.shape

(199797, 21)

### 02. Data Wrangling

In [10]:
# Select only the necessary columns and put them in a list called columns
columns =["DATA_YEAR", "STATE_ABBR", "STATE_NAME", "REGION_NAME", "TOTAL_OFFENDER_COUNT", "OFFENDER_RACE", "VICTIM_COUNT", "OFFENSE_NAME", "BIAS_DESC"]

In [11]:
# Create a subset
hate_crime_vis = df[columns]

In [13]:
# Confirm subset creation
hate_crime_vis.head()

DATA_YEAR STATE_ABBR STATE_NAME REGION_NAME  TOTAL_OFFENDER_COUNT  \
0       1991         AR   Arkansas       South                     1   
1       1991         AR   Arkansas       South                     1   
2       1991         AR   Arkansas       South                     1   
3       1991         AR   Arkansas       South                     1   
4       1991         AR   Arkansas       South                     1   

               OFFENDER_RACE  VICTIM_COUNT  \
0                      White             1   
1  Black or African American             1   
2  Black or African American             1   
3  Black or African American             2   
4  Black or African American             1   

                                        OFFENSE_NAME  \
0                                       Intimidation   
1                                     Simple Assault   
2                                 Aggravated Assault   
3  Aggravated Assault;Destruction/Damage/Vandalis...   
4                                 Aggravated Assault   

                        BIAS_DESC  
0  Anti-Black or African American  
1                      Anti-White  
2  Anti-Black or African American  
3                      Anti-White  
4                      Anti-White

In [14]:
hate_crime_vis.shape

(199797, 9)

### 03. Consistency Checks

In [15]:
# Check for missing values 
hate_crime_vis.isnull().sum()

DATA_YEAR               0
STATE_ABBR              0
STATE_NAME              0
REGION_NAME             0
TOTAL_OFFENDER_COUNT    0
OFFENDER_RACE           0
VICTIM_COUNT            0
OFFENSE_NAME            0
BIAS_DESC               0
dtype: int64

In [16]:
# Check for dupliacte values
dups = hate_crime_vis.duplicated()

In [17]:
dups.shape

(199797,)

##### No duplicates

In [24]:
# Check top 15 types of bias
(df['BIAS_DESC'].value_counts()).sort_values(ascending=False).head(15)

BIAS_DESC
Anti-Black or African American                               68751
Anti-Jewish                                                  25376
Anti-White                                                   23311
Anti-Gay (Male)                                              20257
Anti-Hispanic or Latino                                      12817
Anti-Other Race/Ethnicity/Ancestry                           10018
Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)     6046
Anti-Asian                                                    5893
Anti-Multiple Races, Group                                    4787
Anti-Lesbian (Female)                                         4262
Anti-Islamic (Muslim)                                         3562
Anti-Other Religion                                           3294
Anti-American Indian or Alaska Native                         2158
Anti-Catholic                                                 1422
Anti-Protestant                                     

In [25]:
# Select bias groupings for visualization
hate_crime_visual = hate_crime_vis[(hate_crime_vis['BIAS_DESC'] == 'Anti-Black or African American') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Jewish') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-White') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Gay (Male)') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Hispanic or Latino') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Other Race/Ethnicity/Ancestry') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Asian') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Lesbian (Female)') 
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-Islamic (Muslim)')
         | (hate_crime_vis['BIAS_DESC'] == 'Anti-American Indian/Alaska Native')]

In [28]:
# Confirm subgrouping
hate_crime_visual['BIAS_DESC'].value_counts()

BIAS_DESC
Anti-Black or African American                               68751
Anti-Jewish                                                  25376
Anti-White                                                   23311
Anti-Gay (Male)                                              20257
Anti-Hispanic or Latino                                      12817
Anti-Other Race/Ethnicity/Ancestry                           10018
Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)     6046
Anti-Asian                                                    5893
Anti-Lesbian (Female)                                         4262
Anti-Islamic (Muslim)                                         3562
Name: count, dtype: int64

In [31]:
# Confirm State Abbreviations
hate_crime_visual[['STATE_ABBR','STATE_NAME']].value_counts()

STATE_ABBR  STATE_NAME          
CA          California              30823
NJ          New Jersey              17272
NY          New York                17197
MI          Michigan                10191
MA          Massachusetts            9634
OH          Ohio                     7101
TX          Texas                    6847
WA          Washington               6435
AZ          Arizona                  5502
MD          Maryland                 5101
IL          Illinois                 4970
FL          Florida                  4613
VA          Virginia                 4479
MN          Minnesota                4057
OR          Oregon                   3570
CO          Colorado                 3389
TN          Tennessee                3342
PA          Pennsylvania             3200
CT          Connecticut              3136
MO          Missouri                 2812
KY          Kentucky                 2374
NC          North Carolina           1968
IN          Indiana                  1656
S

In [32]:
# Fix abbreviated name for Nebraska
def nb_check(abbr):
    if abbr == 'NB':
        return 'NE'
    else:
        return abbr

In [35]:
# Create new subset using the above check
by_state = hate_crime_visual
by_state['STATE_ABBR'] = by_state['STATE_ABBR'].apply(nb_check)

C:\Users\Jamie&Ken\AppData\Local\Temp\ipykernel_4772\2992936836.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  by_state['STATE_ABBR'] = by_state['STATE_ABBR'].apply(nb_check)


In [36]:
# Confirm change
by_state[by_state['STATE_ABBR']=='NE']

DATA_YEAR STATE_ABBR STATE_NAME REGION_NAME  TOTAL_OFFENDER_COUNT  \
38113        1996         NE   Nebraska     Midwest                     1   
38114        1996         NE   Nebraska     Midwest                     1   
38115        1996         NE   Nebraska     Midwest                     1   
46476        1997         NE   Nebraska     Midwest                     0   
46477        1997         NE   Nebraska     Midwest                     4   
...           ...        ...        ...         ...                   ...   
196704       2018         NE   Nebraska     Midwest                     0   
196705       2018         NE   Nebraska     Midwest                     0   
196706       2018         NE   Nebraska     Midwest                     0   
196707       2018         NE   Nebraska     Midwest                     0   
196711       2018         NE   Nebraska     Midwest                     0   

                    OFFENDER_RACE  VICTIM_COUNT  \
38113                       White             1   
38114   Black or African American             1   
38115   Black or African American             1   
46476                     Unknown             1   
46477                       White             4   
...                           ...           ...   
196704                    Unknown             2   
196705                    Unknown             1   
196706                    Unknown             1   
196707                    Unknown             2   
196711                    Unknown             1   

                                     OFFENSE_NAME  \
38113                              Simple Assault   
38114                          Aggravated Assault   
38115                          Aggravated Assault   
46476    Destruction/Damage/Vandalism of Property   
46477                          Aggravated Assault   
...                                           ...   
196704  Motor Vehicle Theft;Weapon Law Violations   
196705                  Drug Equipment Violations   
196706   Destruction/Damage/Vandalism of Property   
196707               Burglary/Breaking & Entering   
196711                             Simple Assault   

                                 BIAS_DESC  
38113              Anti-Hispanic or Latino  
38114       Anti-Black or African American  
38115       Anti-Black or African American  
46476                          Anti-Jewish  
46477   Anti-Other Race/Ethnicity/Ancestry  
...                                    ...  
196704                          Anti-White  
196705                          Anti-White  
196706                          Anti-White  
196707                          Anti-White  
196711                          Anti-White  

[868 rows x 9 columns]

In [41]:
# Create hate crime count by state subset to be graphed
by_state = hate_crime_visual.groupby('STATE_NAME').count()

In [39]:
by_state.head()

DATA_YEAR  STATE_ABBR  REGION_NAME  TOTAL_OFFENDER_COUNT  \
STATE_NAME                                                             
Alabama           182         182          182                   182   
Alaska            156         156          156                   156   
Arizona          5502        5502         5502                  5502   
Arkansas          913         913          913                   913   
California      30823       30823        30823                 30823   

            OFFENDER_RACE  VICTIM_COUNT  OFFENSE_NAME  BIAS_DESC  
STATE_NAME                                                        
Alabama               182           182           182        182  
Alaska                156           156           156        156  
Arizona              5502          5502          5502       5502  
Arkansas              913           913           913        913  
California          30823         30823         30823      30823

In [55]:
# insert index
by_state = by_state.reset_index()

In [62]:
# confirm index reset
by_state.head()

STATE_NAME  DATA_YEAR  STATE_ABBR  REGION_NAME  TOTAL_OFFENDER_COUNT  \
0     Alabama        182         182          182                   182   
1      Alaska        156         156          156                   156   
2     Arizona       5502        5502         5502                  5502   
3    Arkansas        913         913          913                   913   
4  California      30823       30823        30823                 30823   

   OFFENDER_RACE  VICTIM_COUNT  OFFENSE_NAME  BIAS_DESC  
0            182           182           182        182  
1            156           156           156        156  
2           5502          5502          5502       5502  
3            913           913           913        913  
4          30823         30823         30823      30823

### 04. Choropleth Map Creation

In [46]:
# Create a data frame with just the states and the values for rating we want plotted

data_to_plot = by_state[['STATE_NAME','VICTIM_COUNT']]
data_to_plot.head()

STATE_NAME  VICTIM_COUNT
0     Alabama           182
1      Alaska           156
2     Arizona          5502
3    Arkansas           913
4  California         30823

In [65]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot,
    columns = ['STATE_NAME', 'VICTIM_COUNT'],
    key_on = 'feature.properties.name', 
    fill_color = 'RdBu'+'_r', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Victims of Hate Crime").add_to(map)
folium.LayerControl().add_to(map)

map

In [64]:
map.save('plot_data.html')

#### This map can be used to answer question #4 Which states have the highest incidence of hate crime: California, New York and New Jersey.

In [53]:
hate_crime_visual.head()

DATA_YEAR STATE_ABBR STATE_NAME REGION_NAME  TOTAL_OFFENDER_COUNT  \
0       1991         AR   Arkansas       South                     1   
1       1991         AR   Arkansas       South                     1   
2       1991         AR   Arkansas       South                     1   
3       1991         AR   Arkansas       South                     1   
4       1991         AR   Arkansas       South                     1   

               OFFENDER_RACE  VICTIM_COUNT  \
0                      White             1   
1  Black or African American             1   
2  Black or African American             1   
3  Black or African American             2   
4  Black or African American             1   

                                        OFFENSE_NAME  \
0                                       Intimidation   
1                                     Simple Assault   
2                                 Aggravated Assault   
3  Aggravated Assault;Destruction/Damage/Vandalis...   
4                                 Aggravated Assault   

                        BIAS_DESC  
0  Anti-Black or African American  
1                      Anti-White  
2  Anti-Black or African American  
3                      Anti-White  
4                      Anti-White

In [54]:
#exporting data
hate_crime_visual.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hate_crimes_wrangled.csv'))

In [63]:
by_state.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hate_crimes_by_state.csv'))